fonte: https://dados.gov.br/dataset/sistema-de-divida-ativa

### Bibliotecas

In [1]:
import pandas as pd
import numpy as np
from google.oauth2 import service_account

In [2]:
!pip install --upgrade tabula-py

  Using cached tabula_py-2.4.0-py3-none-any.whl (12.0 MB)
  Using cached distro-1.7.0-py3-none-any.whl (20 kB)


In [4]:
import tabula # leitor pdf

### Extração

In [11]:
df = tabula.read_pdf('Dicionario+de+Dados+-+Procuradoria.pdf', pages='all')

In [12]:
df = df[0]

In [13]:
df

,Dado,Tipo,Descrição
0,CPF_CNPJ,Numérico,CPF ou CNPJ do titular
1,NaN,NaN,devedor
2,Interessado,Alfanumérico,Nome do titular devedor
3,NumeroDoProcesso,Numérico,Número atribuído ao
4,NaN,NaN,processo de cobrança
5,AnoDoProcesso,Numérico,Ano em que o processo de
6,NaN,NaN,cobrança foi criado
7,Receita,Alfanumérico,Motivação para geração da
8,NaN,NaN,dívida do titular
9,NaN,NaN,(Ex: Taxa Anual por Hectare –


### Transformação

In [14]:
dado = []
tipo = []
descricao = []

In [15]:
len(df)

22

In [16]:
for i in range(0, 22):
    if i+1 >= 22:
        break
    if pd.isna(df['Dado'][i]) == True:
        pass
    elif pd.isna(df['Dado'][i+1]) == True:
        descricao_aux = df['Descrição'][i]
        for j in range(i+1, 22):
            if pd.isna(df['Dado'][j]) == False:
                break
            else:
                descricao_aux = descricao_aux + ' ' + df['Descrição'][j]
        dado.append(df['Dado'][i])
        tipo.append(df['Tipo'][i])
        descricao.append(descricao_aux)
    else:
        dado.append(df['Dado'][i])
        tipo.append(df['Tipo'][i])
        descricao.append(df['Descrição'][i])

In [17]:
df_final = pd.DataFrame({'Dado': dado, 
                         'Tipo': tipo, 
                         'Descrição': descricao})

In [18]:
df_final[df_final.Dado == 'Receita']['Descrição'].values

array(['Motivação para geração da dívida do titular (Ex: Taxa Anual por Hectare – TAH)'],
      dtype=object)

In [19]:
df_final

,Dado,Tipo,Descrição
0,CPF_CNPJ,Numérico,CPF ou CNPJ do titular devedor
1,Interessado,Alfanumérico,Nome do titular devedor
2,NumeroDoProcesso,Numérico,Número atribuído ao processo de cobrança
3,AnoDoProcesso,Numérico,Ano em que o processo de cobrança foi criado
4,Receita,Alfanumérico,Motivação para geração da dívida do titular (E...
5,NumeroCDA,Alfanumérico,Número de inscrição da Certidão de Dívida Ativa
6,ValorAtualizado,Alfanumérico,Valor atualizado para a dívida calculado pelo ...
7,DataAtualização,Data e Hora,Instante em que o campo ‘ValorAtualizado’ teve...
8,StatusDivida,Alfanumérico,Descrição da situação atual do processo de cob...


In [20]:
df_final.rename(columns={'Descrição': 'Descricao'}, inplace=True)

### Carregamento

In [21]:
credentials = service_account.Credentials.from_service_account_file(filename='/home/alex/apps/drivers/ContaServicoBigQuery.json', 
                                                                    scopes=['https://www.googleapis.com/auth/cloud-platform'])

In [22]:
df_final.to_gbq(credentials=credentials, 
                destination_table='curso_etl.etl_pdf', 
                if_exists='replace')